In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from scipy import stats
from scitbx import *
from sklearn.metrics import mean_squared_error

In [10]:
# for fluxnet
timestamp_format = "%Y-%m-%d %H:%M:%S"

out_folder = r"C:\workspace\repositories\fluxlib\data\synthetic_scenario\comparison"
out_full_folder = r"C:\workspace\repositories\fluxlib\data\synthetic_scenario\comparison_full"

mds_flder = r"C:\workspace\repositories\fluxlib\data\synthetic_scenario\mds_out"
flx4mds_flder = r"C:\workspace\repositories\fluxlib\data\synthetic_scenario\fluxnet4mds_csv"
vld_flder = r"C:\workspace\repositories\fluxlib\data\synthetic_scenario\fluxnet4mds_csv_validate"
apl_flder = r"C:\workspace\repositories\fluxlib\data\synthetic_scenario\fluxnet4rfr_apply"

mds_paths = list(Path(mds_flder).glob(r"*.txt"))
x_paths = list(Path(flx4mds_flder).glob(r"*.csv"))
x_names = [p.stem.split("_")[0] for p in x_paths]

vld_paths = list(Path(vld_flder).glob(r"*.csv"))
apl_paths = list(Path(apl_flder).glob(r"*.csv"))

for pm in mds_paths:
    name = pm.stem.split("_")[0]
    idx = x_names.index(name)
    df_x = pd.read_csv(x_paths[idx], index_col = 0)
    df_x.index = df_x.index.map(
        lambda x: datetime.strptime(str(x), timestamp_format)
    )
    mds = pd.read_csv(pm, delimiter = "\t", skiprows = [1]).set_index(df_x.index)[["NEE_f"]]
    mds = mds.replace(-9999, np.nan)
    
    assert vld_paths[idx].stem.split("_")[0] == name == apl_paths[idx].stem.split("_")[0]
    rfr = pd.read_csv(apl_paths[idx]).set_index(df_x.index)[["truth", "estimates"]]
    res = pd.concat([rfr, mds], axis = 1)
    res.to_csv(Path(out_full_folder).joinpath(f"{pm.stem[0: -4]}.csv"))
    
    vld = pd.read_csv(vld_paths[idx], index_col = 0)
    vld.index = vld.index.map(
        lambda x: datetime.strptime(str(x), timestamp_format)
    )
    # print(vld)
    extract_vld = res.loc[vld.index]
    x = extract_vld.dropna()["truth"]
    y1 = extract_vld.dropna()["estimates"]
    y2 = extract_vld.dropna()["NEE_f"]
    #--------------------------------------------
    _, _, r_value, _, _ = stats.linregress(x, y1)
    r2_1 = r_value**2
    mse = mean_squared_error(x, y1)
    rmse_1 = np.sqrt(mse)   
    # -------------------------------------------
    _, _, r_value, _, _ = stats.linregress(x, y2)
    r2_2 = r_value**2
    mse = mean_squared_error(x, y2)
    rmse_2 = np.sqrt(mse)  
    # -------------------------------------------
    
    extract_vld.to_csv(Path(out_folder).joinpath(f"{pm.stem[0: -4]}.csv"))
    print(f"{name}, RFR: {np.round(r2_1, 2)}, {np.round(rmse_1, 2)}; MDS: {np.round(r2_2, 2)}, {np.round(rmse_2, 2)}")
    # break

AR-SLu, RFR: 0.82, 4.29; MDS: 0.77, 4.98
AR-Vir, RFR: 0.81, 6.15; MDS: 0.77, 6.81
AT-Neu, RFR: 0.73, 6.03; MDS: 0.59, 7.86
AU-Ade, RFR: 0.82, 2.66; MDS: 0.74, 3.25
AU-ASM, RFR: 0.86, 0.78; MDS: 0.78, 0.99
AU-Cpr, RFR: 0.72, 1.29; MDS: 0.65, 1.46
AU-Cum, RFR: 0.39, 4.55; MDS: 0.33, 4.82
AU-DaP, RFR: 0.83, 2.74; MDS: 0.73, 3.51
AU-DaS, RFR: 0.7, 2.68; MDS: 0.66, 2.88
AU-Dry, RFR: 0.79, 1.84; MDS: 0.75, 2.03
AU-Emr, RFR: 0.36, 2.44; MDS: 0.23, 2.92
AU-Fog, RFR: 0.8, 1.88; MDS: 0.8, 1.9
AU-Gin, RFR: 0.84, 1.72; MDS: 0.79, 1.94
AU-GWW, RFR: 0.69, 1.05; MDS: 0.62, 1.2
AU-How, RFR: 0.78, 3.51; MDS: 0.74, 3.79
AU-Lox, RFR: 0.86, 4.33; MDS: 0.84, 4.55
AU-RDF, RFR: 0.66, 3.32; MDS: 0.74, 2.93
AU-Rig, RFR: 0.8, 1.72; MDS: 0.65, 2.55
AU-Rob, RFR: 0.76, 3.82; MDS: 0.73, 4.06
AU-Stp, RFR: 0.79, 1.15; MDS: 0.7, 1.38
AU-TTE, RFR: 0.55, 0.69; MDS: 0.54, 0.75
AU-Wac, RFR: 0.36, 8.65; MDS: 0.25, 9.62
AU-Whr, RFR: 0.79, 2.14; MDS: 0.71, 2.53
AU-Wom, RFR: 0.84, 3.4; MDS: 0.76, 4.1
AU-Ync, RFR: 0.3, 1.88; M

In [24]:
# for malaysia
timestamp_format = "%Y-%m-%d %H:%M:%S"

out_folder = r"C:\workspace\repositories\fluxlib\data\malaysia_synthetic_scenario\comparison"
create_all_parents(out_folder)
out_full_folder = r"C:\workspace\repositories\fluxlib\data\malaysia_synthetic_scenario\comparison_full"
create_all_parents(out_full_folder)

mds_flder = r"C:\workspace\repositories\fluxlib\data\malaysia_synthetic_scenario\mds_out"
# flx4mds_flder = r"C:\workspace\repositories\fluxlib\data\malaysia_synthetic_scenario\fluxnet4mds_csv"
vld_flder = r"C:\workspace\repositories\fluxlib\data\malaysia_synthetic_scenario\malaysia4mds_csv_validate"
apl_flder = r"C:\workspace\repositories\fluxlib\data\malaysia_synthetic_scenario\malaysia4rfr_apply"

mds_path = list(Path(mds_flder).glob(r"*.txt"))[0]
name = mds_path.stem.split("_")[0]

vld_path = list(Path(vld_flder).glob(r"*.csv"))[0]
apl_path = list(Path(apl_flder).glob(r"*.csv"))[0]

df_app = pd.read_csv(apl_path, index_col = 0)
df_app.index = df_app.index.map(
    lambda x: datetime.strptime(str(x), timestamp_format)
)
mds = pd.read_csv(mds_path, delimiter = "\t", skiprows = [1]).set_index(df_app.index)[["NEE_f"]]
mds = mds.replace(-9999, np.nan)

rfr = df_app[["truth", "estimates"]]
res = pd.concat([rfr, mds], axis = 1)
res.to_csv(Path(out_full_folder).joinpath(f"{pm.stem[0: -4]}.csv"))

vld = pd.read_csv(vld_path, index_col = 0)
vld.index = vld.index.map(
    lambda x: datetime.strptime(str(x), timestamp_format)
)
# print(vld)
extract_vld = res.loc[vld.index]
x = extract_vld.dropna()["truth"]
y1 = extract_vld.dropna()["estimates"]
y2 = extract_vld.dropna()["NEE_f"]
extract_vld.to_csv(Path(out_folder).joinpath(f"{pm.stem[0: -4]}.csv"))
#--------------------------------------------
_, _, r_value, _, _ = stats.linregress(x, y1)
r2_1 = r_value**2
mse = mean_squared_error(x, y1)
rmse_1 = np.sqrt(mse)   
# -------------------------------------------
_, _, r_value, _, _ = stats.linregress(x, y2)
r2_2 = r_value**2
mse = mean_squared_error(x, y2)
rmse_2 = np.sqrt(mse)  
# -------------------------------------------


print(f"{name}, RFR: {np.round(r2_1, 2)}, {np.round(rmse_1, 2)}; MDS: {np.round(r2_2, 2)}, {np.round(rmse_2, 2)}")
# break

Sebungan, RFR: 0.78, 6.69; MDS: 0.75, 7.12


In [18]:
# for griffin and rothamsted
# griffin need to change time range!

timestamp_format = "%Y-%m-%d %H:%M:%S"

out_folder = r"C:\workspace\repositories\fluxlib\data\griffin_roth_synthetic_scenario\comparison"
create_all_parents(out_folder)
out_full_folder = r"C:\workspace\repositories\fluxlib\data\griffin_roth_synthetic_scenario\comparison_full"
create_all_parents(out_full_folder)

mds_flder = r"C:\workspace\repositories\fluxlib\data\griffin_roth_synthetic_scenario\mds_out"
flx4mds_flder = r"C:\workspace\repositories\fluxlib\data\griffin_roth_synthetic_scenario\groth4mds"
vld_flder = r"C:\workspace\repositories\fluxlib\data\griffin_roth_synthetic_scenario\groth4mds_csv_validate"
apl_flder = r"C:\workspace\repositories\fluxlib\data\griffin_roth_synthetic_scenario\groth4rfr_apply"

mds_paths = list(Path(mds_flder).glob(r"*.txt"))
mds_time_paths = list(Path(flx4mds_flder).glob(r"*.txt"))
vld_paths = list(Path(vld_flder).glob(r"*.csv"))
apl_paths = list(Path(apl_flder).glob(r"*.csv"))

for count, pm in enumerate(mds_paths):
    name = pm.stem[0: -4]
    df_app = pd.read_csv(apl_paths[count], index_col = 0)
    df_app.index = df_app.index.map(
        lambda x: datetime.strptime(str(x), timestamp_format)
    )
    if count == 0:
        print("griffin, manage time range...")
        mds_time = pd.read_csv(mds_time_paths[count], delimiter = "\t", skiprows = [1])[["Year", "DoY", "Hour"]]
        split_time = mds_time['Hour'].astype(str).str.split('.', 1, expand = True)
        mds_time["Hour"] = split_time[0].astype(int)
        mds_time["Minute"] = split_time[1].astype(int) * 6
        mds_time["time"] = mds_time["Year"].astype(str) + "-" + mds_time["DoY"].astype(str) + \
                           " " + mds_time["Hour"].astype(str).str.zfill(2) + ":" + mds_time["Minute"].astype(str).str.zfill(2)
        # print(mds_time)
        mds_time["time"] = mds_time["time"].map(
            lambda x: datetime.strptime(x, r"%Y-%j %H:%M")
        )
        mds_time = mds_time["time"]
        df_app = df_app.loc[mds_time, :]
        # print(mds_time)
    
        mds = pd.read_csv(pm, delimiter = "\t", skiprows = [1]).set_index(mds_time)[["NEE_f"]]
        mds = mds.replace(-9999, np.nan)
    else:
        mds = pd.read_csv(pm, delimiter = "\t", skiprows = [1]).set_index(df_app.index)[["NEE_f"]]
        mds = mds.replace(-9999, np.nan)
    
    rfr = df_app[["truth", "estimates"]]
    res = pd.concat([rfr, mds], axis = 1)
    res.to_csv(Path(out_full_folder).joinpath(f"{pm.stem[0: -4]}.csv"))
    
    
    vld = pd.read_csv(vld_paths[count], index_col = 0)
    vld.index = vld.index.map(
        lambda x: datetime.strptime(str(x), timestamp_format)
    )
    # print(vld)
    extract_vld = res.loc[vld.index]
    x = extract_vld.dropna()["truth"]
    y1 = extract_vld.dropna()["estimates"]
    y2 = extract_vld.dropna()["NEE_f"]
    #--------------------------------------------
    _, _, r_value, _, _ = stats.linregress(x, y1)
    r2_1 = r_value**2
    mse = mean_squared_error(x, y1)
    rmse_1 = np.sqrt(mse)   
    # -------------------------------------------
    _, _, r_value, _, _ = stats.linregress(x, y2)
    r2_2 = r_value**2
    mse = mean_squared_error(x, y2)
    rmse_2 = np.sqrt(mse)  
    # -------------------------------------------
    
    extract_vld.to_csv(Path(out_folder).joinpath(f"{pm.stem[0: -4]}.csv"))
    print(f"{name}, RFR: {np.round(r2_1, 2)}, {np.round(rmse_1, 2)}; MDS: {np.round(r2_2, 2)}, {np.round(rmse_2, 2)}")
    # break

griffin, manage time range...
griffin , RFR: 0.72, 3.62; MDS: 0.03, 14.24
Roth_L , RFR: 0.81, 3.31; MDS: 0.74, 3.81
Roth_N , RFR: 0.25, 11.23; MDS: 0.0, 40.47
Roth_S , RFR: 0.15, 10.68; MDS: 0.05, 65.07
